In [2406]:
from src.routing import create_nodes_dataframe, custom_clustering, find_route, plot_refined_clusters, plot_ind_route, plot_all_cluster_routes

In [2407]:
nodes_df, distance_matrix = create_nodes_dataframe(num_nodes=50, min_work_days=4, home_node_id=0, visiting_interval_min=10, visiting_interval_max=30, max_last_visit=20)

In [2408]:
"""
- Define subset of nodes_df based on node priority and distance to root node (those further away should be included more likely)
- Test Discrete Priority (must be visited this week; must be visited next week; ...)
"""
nodes_df = nodes_df

In [2409]:
nodes_df['angle_to_home']

0       0.000000
1     302.735226
2      81.869898
3     199.025606
4     277.431408
5     346.607502
6     308.418055
7     118.739795
8     290.973493
9     258.023868
10     37.405357
11    225.000000
12    321.483074
13    273.179830
14    135.000000
15    237.907409
16    288.434949
17    315.000000
18    246.571307
19    345.677280
20    233.471145
21    219.173658
22    321.340192
23    272.045408
24     47.726311
25    225.000000
26    294.507405
27    278.130102
28    284.036243
29    210.465545
30    297.897271
31    240.255119
32      0.000000
33    352.874984
34    289.057705
35    272.161079
36     74.357754
37    246.124719
38    359.045159
39    211.504267
40    261.027373
41    248.498566
42     21.974508
43     19.983107
44    236.534621
45    124.695154
46     16.336043
47    214.695154
48    320.648247
49    338.404690
Name: angle_to_home, dtype: float64

In [2410]:
if nodes_df.index[0] == 0:
    nodes_df_copy = nodes_df.drop(0).copy()
        
angles = sorted(nodes_df_copy['angle_to_home'])

# Calculate differences between consecutive angles
diffs = [angles[i + 1] - angles[i] for i in range(len(angles) - 1)]

# Consider the circular difference between the last and first angles
diffs.append(360 - angles[-1] + angles[0])

# Find the largest gap
max_gap = max(diffs)

# Identify the range of the largest gap
start_angle = angles[diffs.index(max_gap)]
end_angle = angles[(diffs.index(max_gap) + 1) % len(angles)]

print(f"Largest gap spans from {start_angle}° to {end_angle}°, covering {max_gap}°.")

Largest gap spans from 135.0° to 199.02560603756868°, covering 64.02560603756868°.


In [2411]:
def custom_clustering(distance_matrix, nodes_df, num_small_clusters, num_large_clusters, overnight_factor, precision, adjustment_speed, home_node_id = 0):
    """
    Custom clustering algorithm that divides nodes into angular clusters.
    Balances clusters between small and large based on the metric calculations.
    Includes the home node at the start of each cluster.
    
    Args:
    - distance_matrix: DataFrame containing distances between nodes.
    - nodes_df: DataFrame containing node data.
    - num_small_clusters: Number of smaller clusters to create.
    - num_large_clusters: Number of larger clusters to create.
    - factor: Adjustment factor used in balancing clusters.
    - precision: Number of iterations for refining the clusters.
    - home_node_id: ID of the home node to be included at the start of each cluster.
    """
    if nodes_df.index[0] == 0:
        nodes_df = nodes_df.drop(0)
    
    clusters = {f'small_{i}': [home_node_id] for i in range(num_small_clusters)}
    clusters.update({f'large_{i}': [home_node_id] for i in range(num_large_clusters)})

    angles = sorted(nodes_df_copy['angle_to_home'])
    diffs = [angles[i + 1] - angles[i] for i in range(len(angles) - 1)]
    diffs.append(360 - angles[-1] + angles[0])

    max_gap = max(diffs)
    start_angle = angles[diffs.index(max_gap)]
    end_angle = angles[(diffs.index(max_gap) + 1) % len(angles)]



    gap_start = angles[diffs.index(max_gap)]
    gap_end = angles[(diffs.index(max_gap) + 1) % len(angles)]
    # Define cluster_start and cluster_end with proper circular handling
    cluster_start = gap_end
    cluster_end = gap_start if gap_start > gap_end else gap_start + 360
    # Normalize the cluster_end to ensure it does not exceed 360
    cluster_end = cluster_end if cluster_end <= 360 else cluster_end - 360

    num_clusters = num_small_clusters + num_large_clusters
    step = (cluster_end - cluster_start) % 360 / num_clusters # step = max_gap / num_clusters
    current_angle = cluster_start
    # max_gap = 360

    # angle_ranges = {f'small_{i}': (i * step, (i + 1) * step) for i in range(num_small_clusters)}
    # angle_ranges.update({f'large_{i}': ((i + num_small_clusters) * step, (i + num_small_clusters + 1) * step) for i in range(num_large_clusters)})

    # BECOMES
    angle_ranges = {}
    for i in range(num_clusters):
        range_start = current_angle
        range_end = (current_angle + step) % 360
        cluster_id = f'small_{i}' if i < num_small_clusters else f'large_{i - num_small_clusters}'
        angle_ranges[cluster_id] = (range_start, range_end)
        current_angle = range_end

    # Iteratively adjust cluster assignments based on the metrics
    for _ in range(precision):
        # Start each iteration by clearing clusters but keeping the home node
        for key in clusters.keys():
            clusters[key] = [home_node_id]

        # Distribute nodes based on (adjusted) angle ranges
        for index, row in nodes_df.iterrows():
            if index == home_node_id:
                continue  # Skip the home node in regular assignments
            node_angle = row['angle_to_home']
            for cluster_id, (cluster_start_angle, cluster_end_angle) in angle_ranges.items():
                
                
                
                if cluster_start_angle <= node_angle < cluster_end_angle or (cluster_start_angle > cluster_end_angle and (node_angle >= cluster_start_angle or node_angle < cluster_end_angle)):
                # if cluster_start_angle <= node_angle < cluster_end_angle:
                    
                    
                    
                    clusters[cluster_id].append(index)
                    break

        # Adjust angles based on cluster metrics
        # removed the current_angle parameter  
        angle_ranges = adjust_angles(clusters, nodes_df, angle_ranges, adjustment_speed)

    return clusters, angle_ranges

def adjust_angles(clusters, nodes_df, angle_ranges, adjustment_speed):    
    total_nodes = len(nodes_df)
    num_clusters = len(clusters)
    desired_size = total_nodes / num_clusters
    current_sizes = {cluster_id: len(indices) for cluster_id, indices in clusters.items()}
    angle_changes = {}
    total_angle_change = 0

    for cluster_id, size in current_sizes.items():
        deviation = size - desired_size
        angle_changes[cluster_id] = -deviation * adjustment_speed
        total_angle_change += angle_changes[cluster_id]

    correction = -total_angle_change / num_clusters

    adjusted_angle_ranges = {}
    for cluster_id, initial_range in angle_ranges.items():
        start_angle, end_angle = initial_range
        angle_width = end_angle - start_angle + angle_changes[cluster_id] + correction
        adjusted_angle_ranges[cluster_id] = ((start_angle + angle_width) % 360, (end_angle + angle_width) % 360)

    return adjusted_angle_ranges

def adjust_angles(clusters, nodes_df, angle_ranges, adjustment_speed):    
    total_nodes = len(nodes_df)
    num_clusters = len(clusters)
    desired_size = total_nodes / num_clusters

    # Calculate current sizes and prepare adjustments
    current_sizes = {cluster_id: len(indices) for cluster_id, indices in clusters.items()}
    angle_changes = {}

    # Calculate total angle change needed based on node distribution
    total_angle_change = 0
    for cluster_id, size in current_sizes.items():
        deviation = size - desired_size
        # Here, angle adjustment could be proportional to the deviation from desired size
        angle_changes[cluster_id] = -deviation * adjustment_speed  # Adjust scaling factor as needed
        total_angle_change += angle_changes[cluster_id]

    # Ensure total angle remains 360 degrees
    correction = -total_angle_change / num_clusters # Potentially CHANGE HERE or below

    # Adjust angles
    adjusted_angle_ranges = {}
    for cluster_id, initial_range in angle_ranges.items():
        start_angle, end_angle = initial_range
        angle_width = (end_angle - start_angle + angle_changes[cluster_id] + correction) % 360
        adjusted_angle_ranges[cluster_id] = (start_angle, (start_angle + angle_width) % 360)
    

    # OLD
    # for cluster_id, initial_range in angle_ranges.items():
    #     # initial start and end angle are accessed
    #     start_angle, end_angle = initial_range
    #     # the new total angle width is defined
    #     angle_width = end_angle - start_angle + angle_changes[cluster_id] + correction
    #     # adjusted angle ranges per cluster ID are stored
    #     adjusted_angle_ranges[cluster_id] = (current_angle, current_angle + angle_width)
    #     # the current_angle is updated
    #     current_angle += angle_width
    
    return adjusted_angle_ranges

In [2412]:
"""
- Prevent clusters from jumping angles when home node is not in the center
- Make large clusters larger
- Adjust angles based on:
    - number of nodes
    - variance and mean of priorities
    - variance and mean distances between nodes
    - max and mean of distance to root
- Respect fixed appointments
"""
adjustment_speed = len(nodes_df) / 100
clusters, angle_ranges = custom_clustering(distance_matrix.values, nodes_df, num_small_clusters=5, num_large_clusters=0, overnight_factor=1.5, precision=100, adjustment_speed=adjustment_speed)

In [2413]:
print([len(c) for c in clusters.values()])

[6, 6, 6, 7, 6]


In [2414]:
plot_refined_clusters(clusters, nodes_df)

In [2415]:
"""
- How to choose max_travel_distance and span_cost_coefficient?
- Define route based on:
    - working hours (implicitly defined by total travel time)
    - opening hours
    - fixed appointments
- Store state and if it was possible to find routes for all solutions add nodes
- Compare routes with and without overnight stays / large clusters
"""
route_lists = find_route(clusters, nodes_df, distance_matrix)
plot_all_cluster_routes(route_lists, nodes_df)